### 1. Importamos librerías y cargamos los datos procesados

In [1]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error

processed_dir = 'data/processed'
train_df = pd.read_csv(os.path.join(processed_dir, 'train.csv'))
test_df  = pd.read_csv(os.path.join(processed_dir, 'test.csv'))


### 2. Declaramos nuestros hiperparámetros y constantes


In [2]:
NUM_ITERATIONS = 10      # iteraciones
MIN_RATING, MAX_RATING = 1.0, 5.0

#### Matriz de valoraciones

In [3]:
# lista de listas (0 donde no hay rating)
num_users = int(max(train_df.user_id.max(), test_df.user_id.max()))
num_items = int(max(train_df.book_id.max(), test_df.book_id.max()))
R = [[0.0]*num_items for _ in range(num_users)]
for _, row in train_df.iterrows():
    u = int(row.user_id) - 1
    i = int(row.book_id) - 1
    R[u][i] = float(row.rating)

### 3. Inicializamos el modelo, hiperparámetros

In [4]:
NUM_FACTORS    = 7       # f
LEARNING_RATE  = 0.001   # γ
REGULARIZATION = 0.1     # λ

### 4. Creamos P y Q con uniformes en [0,1]

In [5]:
random.seed(42)
P = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(num_users)]
Q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(num_items)]

### 5. SGD iterativo

In [ ]:
for it in range(NUM_ITERATIONS):
    print(f"Iteración {it+1}/{NUM_ITERATIONS}")
    updated_P = [row.copy() for row in P]
    updated_Q = [row.copy() for row in Q]

    # Recorremos sólo las valoraciones reales:
    for _, row in train_df.iterrows():
        u = int(row.user_id) - 1
        i = int(row.book_id) - 1
        r_ui = float(row.rating)

        # Cálculo del error
        pred = sum(P[u][k] * Q[i][k] for k in range(NUM_FACTORS))
        e = r_ui - pred

        # Actualización por cada factor latente
        for k in range(NUM_FACTORS):
            p_uk = P[u][k]
            q_ik = Q[i][k]
            grad_p = e * q_ik - REGULARIZATION * p_uk
            grad_q = e * p_uk - REGULARIZATION * q_ik
            updated_P[u][k] += LEARNING_RATE * grad_p
            updated_Q[i][k] += LEARNING_RATE * grad_q

    P, Q = updated_P, updated_Q

Iteración 1/10
Iteración 2/10


### 6. Evaluación del modelo


In [ ]:
def get_regression_metrics(test_df, pred_matrix):
    """devolvemos MAE y RMSE para todas las valoraciones de test_df"""
    y_true = test_df['rating'].values
    y_pred = [
        pred_matrix[int(row.user_id)-1, int(row.book_id)-1]
        for _, row in test_df.iterrows()
    ]
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

pred_matrix = np.dot(np.array(P), np.array(Q).T)
pred_matrix = np.clip(pred_matrix, MIN_RATING, MAX_RATING) # usamos clip para asegurar que todos los ratings queden en el intervalo que queremos

mae, rmse = get_regression_metrics(test_df, pred_matrix)

KeyboardInterrupt: 

#### Métricas de clasificación binaria

Tanto el **RMSE** como el **MAE** son métricas de regresión (distancia a la realidad de la predicción), mientras que el **F1-score** o el **Recall** miden la precisión con que nuestro modelo recomienda algo que, efectivamente, gusta al usuario. Para ello desarrollamos la siguiente función.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def get_binary_metrics(test_df, pred_matrix, threshold=4.0):
    """
    Binariza la tarea en like>=threshold vs dislike<threshold.
    Devuelve precision, recall, f1.
    """
    y_true = (test_df['rating'] >= threshold).astype(int).values
    y_pred = [
        1 if pred_matrix[int(row.user_id)-1, int(row.book_id)-1] >= threshold else 0
        for _, row in test_df.iterrows()
    ]
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)
    return prec, rec, f1

prec, rec, f1 = get_binary_metrics(test_df, pred_matrix, threshold = 3.5)

#### nDCG@K (Normalized Discounted Cumulative Gain)

Para evaluar la **calidad del ranking** de nuestras recomendaciones usamos nDCG@K, que mide:

1. **Relevancia**: asignamos a cada ítem una relevancia real (por ejemplo `1` si la valoración ≥4, `0` en otro caso).  
2. **Posición**: los ítems recomendados en los primeros puestos pesan más que los de atrás, mediante un descuento logarítmico.

- **1.0** indica ranking perfecto (los K ítems más relevantes están en las primeras posiciones).  
- **0.0** indica que ninguno de los K primeros es relevante.

En nuestro experimento usamos **nDCG@10** para comparar KNN, PMF y SVD, asegurando no solo que predecimos bien los ratings (RMSE/MAE), sino también que colocamos primero las recomendaciones que el usuario considera más valiosas.


In [ ]:
def get_user_ndcg(u, pred_matrix, test_df, K=5, rel_col='rel'):
    """
    nDCG@K para el usuario u (0-based).
    test_df debe tener columnas ['user_id','book_id', rel_col].
    rel_col es la relevancia (binaria o graduada).
    """
    user_ratings = test_df[test_df.user_id == (u+1)]
    if user_ratings.empty:
        return None

    true_rels = {
        int(row.book_id)-1: row[rel_col]
        for _, row in user_ratings.iterrows()
    }

    scores = pred_matrix[u]
    # Top-K 
    top_k = np.argsort(scores)[::-1][:K]

    dcg = 0.0
    for rank, item in enumerate(top_k, start=1):
        rel = true_rels.get(item, 0)
        dcg += (2**rel - 1) / np.log2(rank + 1)

    ideal_rels = sorted(true_rels.values(), reverse=True)[:K]
    idcg = sum((2**rel - 1) / np.log2(idx + 1)
               for idx, rel in enumerate(ideal_rels, start=1))

    return dcg / idcg if idcg > 0 else 0.0

def get_ndcg(pred_matrix, test_df, K=5, rel_col='rel'):
    """
    nDCG@K promedio sobre todos los usuarios con al menos una valoración.
    """
    total, count = 0.0, 0
    num_users = pred_matrix.shape[0]
    for u in range(num_users):
        val = get_user_ndcg(u, pred_matrix, test_df, K, rel_col)
        if val is not None:
            total += val
            count += 1
    return (total / count) if count > 0 else 0.0

test_df['rel'] = (test_df.rating >= 4).astype(int)


# nDCG@5
ndcg_5 = get_ndcg(pred_matrix, test_df, K=5, rel_col='rel')
print(f"nDCG@5 = {ndcg_5:.4f}")


## Incluimos un sesgo

Como un 4 no significa lo mismo para 2 usuarios diferentes, incluimos sesgo tanto en los usuarios como en los libros

**SGD iterativo** con sesgos, devolviendo mu y las listas de sesgos

In [ ]:
def biased_train(train_df, P, Q, lr, reg_f, reg_b, epochs):
    """
    P, Q y sesgos mu, b_u, b_i usando SGD.
    Devuelve mu, b_u, b_i, P, Q actualizados.
    """
    # dimensiones
    num_users = int(train_df.user_id.max())
    num_items = int(train_df.book_id.max())
    
    mu  = train_df.rating.mean()
    b_u = np.zeros(num_users)
    b_i = np.zeros(num_items)
    
    for _ in range(epochs):
        for _, row in train_df.iterrows():
            u = int(row.user_id) - 1
            i = int(row.book_id) - 1
            r = row.rating
            
            pred = mu + b_u[u] + b_i[i] + P[u].dot(Q[i])
            err  = r - pred
            
            # actualizar sesgos
            b_u[u] += lr * (err - reg_b * b_u[u])
            b_i[i] += lr * (err - reg_b * b_i[i])
            
            # actualizar factores
            for k in range(P.shape[1]):
                p_uk = P[u, k]
                q_ik = Q[i, k]
                P[u, k] += lr * (err * q_ik - reg_f * p_uk)
                Q[i, k] += lr * (err * p_uk - reg_f * q_ik)
    
    return mu, b_u, b_i, P, Q


#### Matriz de predicciones

In [ ]:
def biased_pred_matrix(mu, b_u, b_i, P, Q, min_r=1.0, max_r=5.0):
    """
    Devuelve pred_matrix de forma (NUM_USERS, NUM_ITEMS) con sesgos.
    """
    base = P.dot(Q.T)                 # (NUM_USERS, NUM_ITEMS)
    preds = mu + b_u[:, None] + b_i[None, :] + base
    return np.clip(preds, min_r, max_r)

In [ ]:
mu, b_u, b_i, P, Q = biased_train(
    train_df,
    P, Q,
    lr=0.001,
    reg_f=0.1,
    reg_b=0.01,
    epochs=20
)

pred_matrix = biased_pred_matrix(mu, b_u, b_i, P, Q, 1.0, 5.0)